In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime, math

In [20]:
busArrivals = pd.read_csv("./data/Arrivals_M100.csv")
justTimes = busArrivals.loc[:,['RecordedAtTime','VehicleRef']].sort_values(by=['VehicleRef','RecordedAtTime'])
justTimes['RecordedAtTime'] = pd.to_datetime(justTimes['RecordedAtTime'])
#justTimes['timeDelta'] = justTimes['RecordedAtTime'].diff()
justTimes = justTimes.reset_index()
justTimes.head(20)

,index,RecordedAtTime,VehicleRef
0,4226,2017-08-23 10:01:19,NYCT_1202
1,4230,2017-08-23 10:11:48,NYCT_1202
2,4277,2017-08-23 12:51:31,NYCT_1202
3,4280,2017-08-23 13:02:02,NYCT_1202
4,4282,2017-08-23 13:11:27,NYCT_1202
5,4288,2017-08-23 13:51:35,NYCT_1202
6,4330,2017-08-23 16:12:27,NYCT_1202
7,4340,2017-08-23 16:52:25,NYCT_1202
8,1160,2017-08-07 09:33:42,NYCT_1206
9,1169,2017-08-07 10:13:36,NYCT_1206


In [29]:

for index, row in justTimes.iterrows():
    justTimes.loc[index,'TimeDelta'] = justTimes[index+1,'RecordedAtTime'] - justTimes[index,'RecordedAtTime']
    # row['timeDelta'] = row.loc[index+1,'RecordedAtTime'] - row.loc[index,'RecordedAtTime']
justTimes.head()

KeyError: (1, 'RecordedAtTime')